In [2]:
import os
from neo4j import GraphDatabase
from dotenv import load_dotenv

load_dotenv()

graph = GraphDatabase.driver(
	os.environ['NEO4J_URI'],
	auth=(os.environ['NEO4J_USER'], os.environ['NEO4J_PWD'])
)
database = os.environ['NEO4J_DATABASE']

with graph.session(database=database) as session:
	records = session.execute_read(
		lambda tx: tx.run('MATCH (b:Bird)-[:HAS_FACT]->(d:Fact) ORDER BY RAND() RETURN b.name AS name, d.text AS text LIMIT 20').data()
	)
records

[{'name': 'Pale Mountain-Pigeon', 'text': 'Monotypic.'},
 {'name': 'Angola Swallow',
  'text': 'Conservation status on BirdlifeLCLeast ConcernNot globally threatened. Rare to locally common. Uncommon in W of range; in E, commoner W of Rift Valley than E of it, but locally common in E Tanzania. Nests mainly in natural sites, but may be increasing its use of artificial ones; range possibly expanded relatively recently N into Gabon.'},
 {'name': 'Fuerteventura Stonechat', 'text': 'Fuerteventura, in E Canary Is.'},
 {'name': 'Polynesian Triller',
  'text': 'SUBSPECIESLalage maculosa modestaScientific name definitionsDistributionN and C Vanuatu (Espiritu Santo, Malo, Malakula, Pauuma, Epi, Tongoa and Emae).SUBSPECIESLalage maculosa ultimaScientific name definitionsDistributionC Vanuatu (Éfaté I).SUBSPECIESLalage maculosa melanopygiaScientific name definitionsDistributionSanta Cruz Is (Utupa, Nendo).SUBSPECIESLalage maculosa vanikorensisScientific name definitionsDistributionSanta Cruz Is (V

In [3]:
import re
from typing import List, Annotated
from pydantic import BaseModel, BeforeValidator
from enum import Enum

import instructor
import pandas as pd

client = instructor.from_provider(
    'ollama/gpt-oss:20b',
	base_url='http://localhost:11434/v1',
	mode=instructor.Mode.JSON,
)

seed = 42

instruct_params = {
	'max_retries': 2,
	'timeout': 15.0,
	'extra_body': {
		'options': {
			'temperature': 0.0,
			'seed': 42,
			'top_k': 1,
			'num_predict': 50
		}
	}
}


class Continent(str, Enum):
	NORTH_AMERICA = 'North America'
	SOUTH_AMERICA = 'South America'
	EUROPE = 'Europe'
	ASIA = 'Asia'
	AFRICA = 'Africa'
	OCEANIA = 'Oceania'
	ANTARCTICA = 'Antarctica'

class Continents(BaseModel):
    continents: List[Continent]

def get_continents(text: str):
	try:
		resp = client.create(
			response_model=Continents,
			messages=[
				{'role': 'system', 'content': (
					'Your goal is to determine which continents a Bird could be located in, given a text passage describing a fact about the bird. '
					'Only return continents that are either directly referenced in the text, or that have countries or regions within that continent that are directly referenced in the text. '
					'Only return continents that that describe where the bird can be found in its natural habitat. '
				)},
				{'role': 'user', 'content': f'Text: *{text}* DO NOT EXPLAIN'}
			],
			**instruct_params
		)
		return [ c.value for c in resp.continents ]
	except Exception as e:
		print(f'CONTINENTS ERR: {e}')
		return []


c_df = pd.read_csv('../data/countries_by_continent.csv')
countries = c_df.Country.values.tolist()

def clean_alpha_only(val: str) -> str:
	return re.sub(r'\s+', ' ', re.sub(r'[^a-zA-Z]', ' ', val)).strip()
    
Country = Enum('Country', { clean_alpha_only(country).upper(): clean_alpha_only(country) for country in countries }, type=str)

class Countries(BaseModel):
    countries: List[
        Annotated[
            Country,
            BeforeValidator(clean_alpha_only)
        ]
	]

def get_countries(text: str):
	try:
		countries = client.create(
			response_model=Countries,
			messages=[
				{'role': 'system', 'content': (
					'Your goal is to determine which countries a Bird could be located in, given a text passage describing a fact about the bird. '
					'Only return countries that are either directly referenced in the text, or that have regions within that country that are directly referenced in the text. '
					'Only return countries that that describe where the bird can be found in its natural habitat. '
				)},
				{'role': 'user', 'content': f'Text: *{text}* DO NOT EXPLAIN'}
			],
			**instruct_params
		)
		return [ c.value for c in countries.countries ]
	except Exception as e:
		print(f'COUNTRIES ERR: {e}')
		return []
		

class Regions(BaseModel):
    regions: List[str]
 
def get_regions(text: str):
	try:
		resp = client.create(
			response_model=Regions,
			messages=[
				{'role': 'system', 'content': (
					'Your goal is to extract all geographic locations (e.g. mountain ranges, deserts, forests, etc.) that a bird could be located in, given a text passage describing a fact about the bird. '
					'Geographic locations do not include countries, full continents, cities, or man-made structures. '
					'Return geographic locations without abbreviations, e.g. "Ural Mountains" instead of "Ural mtns", "Cooke Island" instead of "Cooke I", etc. '
					'Only return geographic locations that are directly referenced in the text and that describe where the bird can be found in its natural habitat. '
				)},
				{'role': 'user', 'content': f'Text: *{text}* DO NOT EXPLAIN'}
			],
			**instruct_params
		)
		return [ 
			' '.join([ w.capitalize() for w in r.split() ])
			for r in resp.regions 
        ]
	except Exception as e:
		print(f'REGIONS ERR: {e}')
		return []


for record in records:
#for record in [{'text': 'New Caledonian Buttonquail has traditionally been considered conspecific with Painted Buttonquail (Turnix varius), but recognized as very distinctive in HBW and separated in one recent list (2). In their assessment of the two taxa, del Hoyo and Collar (3), using the Tobias et al. (4) criteria, from which the numbers in brackets are derived, found that they differ in several plumage and morphometric characters. The comparison is based on one specimen of New Caledonian Buttonquail, a male with no specific locality and undated, but registered in the Natural History Museum in London in 1889: it differs from males of Painted Buttonquail in its smaller size (wing 80 mm versus 101.5 in one published sample (5), and bill 14.1 mm, tarsus 20 mm, and tail 39 mm) [allow 3]; dorsal and rump feathers mostly black versus black with rusty barring, without the rust color predominating on the mantle, and with proportionately more black markings in wing-coverts [2]; breast barred blackish-and-buff with some pale grey bases versus pale gray with buff blades with narrow blackish edges [2].'}]:
	print(f'TEXT: {record['text']}\n')
	continents = get_continents(record['text'])	
	print(f'Continents: {continents}')
	countries = get_countries(record['text'])
	print(f'Countries: {countries}')
	continents_from_countries = c_df[c_df.Country.isin(countries)].Continent.unique().tolist()
	print(f'Continents from Countries: {continents_from_countries}')
	regions = get_regions(record['text'])
	print(f'Regions: {regions}')
	print('\n')

2026-01-02 16:53:08 INFO instructor.auto_client: Initializing ollama provider with model gpt-oss:20b
2026-01-02 16:53:08 INFO instructor.auto_client: Client initialized


TEXT: Monotypic.

Continents: []
Countries: []
Continents from Countries: []
Regions: []


TEXT: Conservation status on BirdlifeLCLeast ConcernNot globally threatened. Rare to locally common. Uncommon in W of range; in E, commoner W of Rift Valley than E of it, but locally common in E Tanzania. Nests mainly in natural sites, but may be increasing its use of artificial ones; range possibly expanded relatively recently N into Gabon.

Continents: ['Africa']
Countries: ['Gabon', 'Tanzania']
Continents from Countries: ['Africa']
Regions: ['Rift Valley']


TEXT: Fuerteventura, in E Canary Is.

Continents: ['Europe']
Countries: ['Spain']
Continents from Countries: ['Europe']
Regions: ['Fuerteventura', 'East Canary Islands']


TEXT: SUBSPECIESLalage maculosa modestaScientific name definitionsDistributionN and C Vanuatu (Espiritu Santo, Malo, Malakula, Pauuma, Epi, Tongoa and Emae).SUBSPECIESLalage maculosa ultimaScientific name definitionsDistributionC Vanuatu (Éfaté I).SUBSPECIESLalage maculo

KeyboardInterrupt: 

In [5]:
from typing import (
	Union,
    Dict
)
import os
import warnings
from neo4j import GraphDatabase
from tqdm.notebook import tqdm
import logging
from concurrent.futures import ThreadPoolExecutor, as_completed

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s %(levelname)s %(name)s: %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
    force=True,  # reset handlers if notebook re-runs
)
logging.getLogger("httpx").setLevel(logging.WARNING)
logging.getLogger("httpcore").setLevel(logging.WARNING)

logger = logging.getLogger("notebook")

warnings.filterwarnings("ignore", module="neo4j")
logging.getLogger("neo4j").setLevel(logging.CRITICAL)

from dotenv import load_dotenv

load_dotenv()

graph = GraphDatabase.driver(
	os.environ['NEO4J_URI'],
	auth=(os.environ['NEO4J_USER'], os.environ['NEO4J_PWD']),
	warn_notification_severity="OFF"
)
database = os.environ['NEO4J_DATABASE']

idx = 483750
batch_size = 200
num_processed = 0
total_fact_count = 101292


def get_batch(start_idx: int) -> Dict[str, str]:
	with graph.session(database=database) as session:
		return session.execute_read(
			lambda tx: tx.run('''
				MATCH (f:Fact)
				WHERE id(f) > $idx
				ORDER BY id(f)
				RETURN id(f) as f_id, f.text as f_text
				LIMIT $batch_size
			''', idx=start_idx, batch_size=batch_size).data()
		)


records = get_batch(idx)

with tqdm(total=total_fact_count, unit="rec", desc="Processing") as pbar:
	while records:
		with graph.session(database=database) as session:
			for record in records:
				f_id = record['f_id']
				text = record['f_text']
				
				continents = get_continents(text)
				countries = get_countries(text)
				regions = get_regions(text)

				session.execute_write(
					lambda tx: tx.run('''
						MATCH (b:Bird)-[:HAS_FACT]->(f:Fact) WHERE id(f) = $f_id

						FOREACH (continent IN $continents |
							MERGE (c:Continent {name: continent})
							CREATE (b)-[:IN_CONTINENT]->(c)
						)

						FOREACH (country IN $countries |
							MERGE (cn:Country {name: country})
							CREATE (b)-[:IN_COUNTRY]->(cn)
      					)

						FOREACH (region IN $regions |
							MERGE (r:Region {name: region})
							CREATE (b)-[:IN_REGION]->(r)
      					)
						''', 
						f_id=f_id, 
						continents=continents,
						countries=countries,
						regions=regions
                    )
				)

				pbar.update()

		num_processed += len(records)
		idx = max(records, key=lambda r: r['f_id'])['f_id']
		logger.info(f'Processed {num_processed} Entries, MAX IDX {idx}')
		records = get_batch(idx)

/tmp/ipykernel_82280/1224420708.py:30: PreviewWarning: notification warnings are a preview feature. It might be changed without following the deprecation policy. See also https://github.com/neo4j/neo4j-python-driver/wiki/preview-features.
  graph = GraphDatabase.driver(


Processing:   0%|          | 0/101292 [00:00<?, ?rec/s]

2026-01-02 17:04:00 INFO openai._base_client: Retrying request to /chat/completions in 0.471201 seconds


COUNTRIES ERR: <failed_attempts>

<generation number="1">
<exception>
    2 validation errors for Countries
countries.6
  Input should be 'Algeria', 'Angola', 'Benin', 'Botswana', 'Burkina', 'Burundi', 'Cameroon', 'Cape Verde', 'Central African Republic', 'Chad', 'Comoros', 'Congo', 'Democratic Republic of Congo', 'Djibouti', 'Egypt', 'Equatorial Guinea', 'Eritrea', 'Ethiopia', 'Gabon', 'Gambia', 'Ghana', 'Guinea', 'Guinea Bissau', 'Ivory Coast', 'Kenya', 'Lesotho', 'Liberia', 'Libya', 'Madagascar', 'Malawi', 'Mali', 'Mauritania', 'Mauritius', 'Morocco', 'Mozambique', 'Namibia', 'Niger', 'Nigeria', 'Rwanda', 'Sao Tome and Principe', 'Senegal', 'Seychelles', 'Sierra Leone', 'Somalia', 'South Africa', 'South Sudan', 'Sudan', 'Swaziland', 'Tanzania', 'Togo', 'Tunisia', 'Uganda', 'Zambia', 'Zimbabwe', 'Afghanistan', 'Bahrain', 'Bangladesh', 'Bhutan', 'Brunei', 'Burma Myanmar', 'Cambodia', 'China', 'East Timor', 'Hong Kong', 'India', 'Indonesia', 'Iran', 'Iraq', 'Israel', 'Japan', 'Jordan',

2026-01-02 17:04:22 INFO openai._base_client: Retrying request to /chat/completions in 0.421952 seconds


...

2026-01-04 14:38:20 INFO notebook: Processed 23400 Entries, MAX IDX 600721
2026-01-04 14:39:34 INFO openai._base_client: Retrying request to /chat/completions in 0.458529 seconds
2026-01-04 14:48:06 INFO openai._base_client: Retrying request to /chat/completions in 0.495949 seconds
2026-01-04 14:56:12 INFO openai._base_client: Retrying request to /chat/completions in 0.477550 seconds
2026-01-04 15:02:59 INFO notebook: Processed 23600 Entries, MAX IDX 602491
2026-01-04 15:14:13 INFO openai._base_client: Retrying request to /chat/completions in 0.424055 seconds
2026-01-04 15:24:23 INFO notebook: Processed 23800 Entries, MAX IDX 605197


COUNTRIES ERR: <failed_attempts>

<generation number="1">
<exception>
    1 validation error for Countries
countries.9
  Input should be 'Algeria', 'Angola', 'Benin', 'Botswana', 'Burkina', 'Burundi', 'Cameroon', 'Cape Verde', 'Central African Republic', 'Chad', 'Comoros', 'Congo', 'Democratic Republic of Congo', 'Djibouti', 'Egypt', 'Equatorial Guinea', 'Eritrea', 'Ethiopia', 'Gabon', 'Gambia', 'Ghana', 'Guinea', 'Guinea Bissau', 'Ivory Coast', 'Kenya', 'Lesotho', 'Liberia', 'Libya', 'Madagascar', 'Malawi', 'Mali', 'Mauritania', 'Mauritius', 'Morocco', 'Mozambique', 'Namibia', 'Niger', 'Nigeria', 'Rwanda', 'Sao Tome and Principe', 'Senegal', 'Seychelles', 'Sierra Leone', 'Somalia', 'South Africa', 'South Sudan', 'Sudan', 'Swaziland', 'Tanzania', 'Togo', 'Tunisia', 'Uganda', 'Zambia', 'Zimbabwe', 'Afghanistan', 'Bahrain', 'Bangladesh', 'Bhutan', 'Brunei', 'Burma Myanmar', 'Cambodia', 'China', 'East Timor', 'Hong Kong', 'India', 'Indonesia', 'Iran', 'Iraq', 'Israel', 'Japan', 'Jordan', 

2026-01-04 15:25:53 INFO openai._base_client: Retrying request to /chat/completions in 0.408142 seconds
2026-01-04 15:26:08 INFO openai._base_client: Retrying request to /chat/completions in 0.780632 seconds
2026-01-04 15:46:59 INFO notebook: Processed 24000 Entries, MAX IDX 611615
2026-01-04 15:52:17 INFO openai._base_client: Retrying request to /chat/completions in 0.491983 seconds
2026-01-04 15:52:38 INFO openai._base_client: Retrying request to /chat/completions in 0.475829 seconds
2026-01-04 15:59:09 INFO openai._base_client: Retrying request to /chat/completions in 0.487881 seconds
2026-01-04 16:08:45 INFO notebook: Processed 24200 Entries, MAX IDX 614001
2026-01-04 16:31:31 INFO notebook: Processed 24400 Entries, MAX IDX 615068
2026-01-04 16:36:15 INFO openai._base_client: Retrying request to /chat/completions in 0.467359 seconds
2026-01-04 16:36:30 INFO openai._base_client: Retrying request to /chat/completions in 0.954013 seconds
2026-01-04 16:40:37 INFO openai._base_client: R

REGIONS ERR: <failed_attempts>

<generation number="1">
<exception>
    Request timed out.
</exception>
<completion>
    None
</completion>
</generation>

</failed_attempts>

<last_exception>
    Request timed out.
</last_exception>


2026-01-04 21:20:52 INFO notebook: Processed 27000 Entries, MAX IDX 640232
2026-01-04 21:42:35 INFO notebook: Processed 27200 Entries, MAX IDX 641248
2026-01-04 21:47:46 INFO openai._base_client: Retrying request to /chat/completions in 0.407143 seconds
2026-01-04 22:02:11 INFO openai._base_client: Retrying request to /chat/completions in 0.440595 seconds
2026-01-04 22:04:03 INFO notebook: Processed 27400 Entries, MAX IDX 642938
2026-01-04 22:10:43 INFO openai._base_client: Retrying request to /chat/completions in 0.468574 seconds
2026-01-04 22:25:21 INFO notebook: Processed 27600 Entries, MAX IDX 643690
2026-01-04 22:45:50 INFO notebook: Processed 27800 Entries, MAX IDX 644732
2026-01-04 23:07:38 INFO notebook: Processed 28000 Entries, MAX IDX 645467
2026-01-04 23:29:49 INFO notebook: Processed 28200 Entries, MAX IDX 646512
2026-01-04 23:39:24 INFO openai._base_client: Retrying request to /chat/completions in 0.415437 seconds
2026-01-04 23:39:58 INFO openai._base_client: Retrying requ